In [1]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import geopandas as gp
import numpy as np


In [2]:
surplus = pd.read_csv('data/surplus.csv')
distance_matrix = pd.DataFrame([],columns=['county_A','county_B','distance'])
#surplus = surplus[:10]

In [3]:
surplus.head()

,FIPS,VEG_ACRES07,FRESHVEG_ACRES07,GHVEG_FARMS07,VEG_PRODUCTION_TONS07,State_y,County Name,"Population Estimate, 2007",VEG_CONSUMPTION_TONS07,SURPLUS,COUNTY,geometry,lon,lat
0,1001,NaN,NaN,NaN,0.000000,AL,Autauga,49834,10340.5550,-10340.555000,1001,"POLYGON ((-86.524693 32.707057, -86.524434 32....",-86.642746,32.534927
1,1003,NaN,NaN,NaN,0.000000,AL,Baldwin,172815,35859.1125,-35859.112500,1003,"POLYGON ((-87.4124710380697 30.573861767419, -...",-87.722563,30.727574
2,1005,32.0,30.0,0.0,432.568293,AL,Barbour,29736,6170.2200,-5737.651707,1005,"POLYGON ((-85.1328473916616 31.8003700328662, ...",-85.393229,31.869561
3,1007,NaN,NaN,NaN,0.000000,AL,Bibb,21485,4458.1375,-4458.137500,1007,"POLYGON ((-87.116315 32.835597, -87.155294 32....",-87.126466,32.998632
4,1009,585.0,581.0,2.0,8377.405946,AL,Blount,56866,11799.6950,-3422.289054,1009,"POLYGON ((-86.73121499999999 34.014697, -86.72...",-86.567382,33.980870


In [4]:
def distance_geog(lat1,lon1,lat2,lon2):
    # http://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude-python
    # approximate radius of earth in miles
    R = 3959.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [5]:
for index,county_A in surplus.iterrows():
    for index,county_B in surplus.iterrows():
        d_g = distance_geog(county_A['lat'],county_A['lon'],county_B['lat'],county_B['lon'])
        #distance_matrix = distance_matrix.append(pd.DataFrame([county_A['County Name'],county_B['County Name'],d_g],columns=['county_A','county_B','distance']))
        cl = {'county_A': county_A['County Name'],'county_B': county_B['County Name'],'distance': d_g}
        #print(cl.values())
        distance_matrix = distance_matrix.append(pd.DataFrame(data=np.array(cl.values()).reshape(1,3),columns=['county_A','county_B','distance']))
        
        #distance_matrix['county_B'].append(county_B['County Name'])
        #distance_matrix['distance'].append(distance_geog(county_A['lat'],county_A['lon'],county_B['lat'],county_B['lon']))
            #distance_matrix['county_A'] = county_A['County Name']
            #distance_matrix['county_B'] = county_B['County Name']
            #distance_matrix['distance'] = distance_geog(county_A['lat'],county_A['lon'],county_B['lat'],county_B['lon'])
            

KeyboardInterrupt: 

In [6]:
surplus['centroid'] = surplus['geometry'].centroid

AttributeError: 'Series' object has no attribute 'centroid'